In [7]:
#IMPORTS

import pandas as pd
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns


In [5]:
full_analysis_df = pd.read_excel('c:/Users/u1172791/COH files/Data_2025/COMBINED_TEXT_jan-june_pre_in-gallery_post_emotions_renamed.xlsx')
emotions_mean_by_ida = pd.read_excel('c:/Users/u1172791/COH files/Data_2025/emotion_means_by_id.xlsx')
display(emotions_mean_by_ida)

,ida,afraid_scared,angry_frustrated,anxious,guilty,happy,hopeless,indifferent,inspired,interested,numb,optimistic_hopeful,sad,stressed
0,24012401BF,1.00,0.75,1.00,0.50,0.75,0.25,0.25,1.00,1.50,0.00,1.25,0.75,0.75
1,24012401MG,0.00,0.25,0.25,0.00,0.50,0.25,0.25,0.75,2.25,0.50,1.75,0.50,0.00
2,24012401PM,0.75,1.25,1.00,1.00,1.00,0.25,0.50,1.50,2.00,0.00,2.00,1.00,1.00
3,24012601LS,0.75,1.00,0.00,0.75,1.00,0.50,0.00,1.00,2.00,1.00,2.50,1.00,1.00
4,24012601RS,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,2.00,0.00,0.50,0.25,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,24040610EC,0.00,0.00,0.25,0.00,1.50,0.00,0.50,1.00,2.25,0.00,1.75,0.25,0.50
180,24040610MC,1.00,0.75,0.50,0.00,0.50,1.00,0.75,0.25,1.50,0.00,1.00,1.00,0.50
181,24040610OW,1.50,1.25,1.25,1.00,0.75,1.50,0.50,0.75,2.25,1.00,1.00,1.25,1.00
182,24040611KS,1.25,1.50,0.50,0.25,1.50,1.50,0.25,1.50,2.50,0.25,1.50,1.00,0.25


In [10]:
# Run both Pearson and Spearman correlations for comparison

# get emotion columns
emotion_cols = [col for col in emotions_mean_by_ida.columns if col != 'ida']

#merge age into same dataframe
emotions_with_age = emotions_mean_by_ida.merge(
    full_analysis_df[['ida', 'age']], 
    on='ida', 
    how='left'
)

#loop through all emotions and run correlation
results = []
for emotion in emotion_cols:
    pearson = stats.pearsonr(emotions_with_age['age'], emotions_with_age[emotion])
    spearman = stats.spearmanr(emotions_with_age['age'], emotions_with_age[emotion])

    results.append({
        'emotion': emotion,
        'pearson_r': pearson[0],
        'pearson_p': pearson[1],
        'spearman_r': spearman[0],
        'spearman_p': spearman[1]
    })

results_df = pd.DataFrame(results)
display(results_df)   # *****No significant results****